In [1]:
from src.data.components.captions import create_simple_caption


In [2]:
import os
os.chdir('..')
os.getcwd()


%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd

df_bioclim_lc = pd.read_csv('data/model_ready/s2bms_presence_with_aux_data.csv')

# create_simple_caption()

In [8]:
caption = create_simple_caption(df_bioclim_lc.iloc[0], include_bioclim=False)
caption

'Location with pastures (46%), non-irrigated arable land (34%) and mixed forest (19%).'

In [6]:
from transformers import CLIPProcessor, CLIPModel, CLIPVisionModel, CLIPTextModel
import torch
_model = CLIPModel.from_pretrained('openai/clip-vit-large-patch14')
_processor = CLIPProcessor.from_pretrained('openai/clip-vit-large-patch14')

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/905 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [37]:
_model.config.text_config

CLIPTextConfig {
  "attention_dropout": 0.0,
  "bos_token_id": 49406,
  "dropout": 0.0,
  "dtype": "float32",
  "eos_token_id": 49407,
  "hidden_act": "quick_gelu",
  "hidden_size": 768,
  "initializer_factor": 1.0,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 77,
  "model_type": "clip_text_model",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "projection_dim": 768,
  "transformers_version": "4.57.3",
  "vocab_size": 49408
}

In [26]:
## get text embeddings
text_inputs = _processor(text=['This is description'], return_tensors="pt", padding=True)
text_inputs, text_inputs.input_ids.shape

({'input_ids': tensor([[49406,   589,   533, 13951, 49407]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])},
 torch.Size([1, 5]))

In [28]:
text_outputs = _model.text_model(**text_inputs)
text_outputs

BaseModelOutputWithPooling(last_hidden_state=tensor([[[-0.3884,  0.0229, -0.0522,  ..., -0.4899, -0.3066,  0.0675],
         [-0.0614, -1.2984,  0.8278,  ..., -0.3053,  0.1875,  0.4740],
         [-0.1095,  0.0389,  1.7066,  ..., -0.7612, -0.7950,  0.1629],
         [-0.9350,  0.6179,  1.3318,  ...,  0.0647,  0.0596,  1.4852],
         [-0.3359, -0.0477,  1.8262,  ..., -1.4522, -0.2459,  1.0320]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-3.3590e-01, -4.7680e-02,  1.8262e+00, -2.1436e-01,  1.0580e+00,
          3.4331e-02, -1.2294e+00, -1.2943e+00, -3.6784e-01,  4.1822e-01,
          1.2675e-01,  1.4072e+00, -4.5215e-01,  4.9533e-01,  8.1702e-01,
          7.7681e-02, -5.3146e-01,  1.1084e+00,  5.0339e-01,  1.0930e+00,
         -1.2449e+00, -1.2535e-01,  1.1387e+00,  9.0477e-01, -1.6505e-01,
         -8.8759e-01, -2.6993e-01, -1.3265e+00, -1.2214e-01,  3.5807e-01,
         -1.5460e+00, -5.4894e-01,  1.1861e-01, -1.1337e+00, -8.1010e-01,
         -1.8199e-01, 

In [27]:
text_outputs = _model.text_model(**text_inputs)
text_embeds = text_outputs[1]
text_embeds.shape

torch.Size([1, 768])

In [24]:
text_embeds = _model.text_projection(text_embeds)
print(text_embeds.shape)

torch.Size([1, 768])


In [25]:
text_embeds = text_embeds / text_embeds.norm(dim=-1, keepdim=True)